In [1]:
from os import getcwd
import os.path as op
from glob import glob

In [2]:
from clients.utils import create_allocation_table
from hardware.plates import create_plates_from_allocation_table

expt_name = 'A81_E217'
allocation_table = create_allocation_table(expt_name)
expt_plates = create_plates_from_allocation_table(allocation_table)

In [3]:
from hardware.qpcr import QpcrDataFile

id_qpcr_plate = 'A81_E217_1'
cwd = getcwd()
nav_root = op.abspath(op.join(cwd, op.pardir))
data_folder = op.join(nav_root, 'vanilla', 'tests', 'data')
f = op.join(data_folder, id_qpcr_plate + '.xls')

qpcr_data = QpcrDataFile(f)
id_qpcr_inst_plate = qpcr_data.get_data_by_well()

C:\Users\chris.icely\AppData\Local\conda\conda\envs\jupyter\lib\site-packages\pandas\core\ops.py:816: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  result = getattr(x, name)(y)


In [4]:
from hardware.labchip import LabChip

lc_plate = '20180103_B'
fds = glob(op.join(data_folder, '*' + lc_plate + '*.csv'))

files = {}
for f in fds:
    if 'PeakTable' in f:
        files['peak_file'] = f
    elif 'RawTable' in f:
        files['raw_file'] = f
    elif 'WellTable' in f:
        
        files['well_file'] = f
lc = LabChip(**files)
lc_inst_plate = lc.get_data_by_well()

In [5]:
assays = {
    'Kp_KPC_3.x_KPC19_KPC22': 343,
    'Kp_KPC_4.x_KPC19_KPC23': 352,
    'Kp_KPC_5.x_KPC19_KPC24': 396,
    'Kp_KPC_6.x_KPC20_KPC22': 332,
    'Kp_KPC_x.10_KPC26_KPC29': 199,
    'Kp_KPC_x.9_KPC26_KPC28': 172}

In [7]:
from clients.expt_recipes.common.model_builders import (build_id_qpcr_constituents,
                                                        build_labchip_datas_from_inst_data)
from clients.expt_recipes.vanilla.models import IdConstituents
from clients.expt_recipes.vanilla.model_builders import (build_id_qpcr_datas_from_inst_data,
                                                         create_vanilla_groupings,
                                                         get_wells_by_id_assay)
from clients.expt_recipes.common.utils import create_id_qpcr_lc_mapping, get_lc_dilutions
from clients.expt_recipes.vanilla.models import VanillaMasterTable

In [8]:
%%time
id_plate_constituents = build_id_qpcr_constituents(
    expt_plates[id_qpcr_plate], expt_plates, IdConstituents)

wells_by_id_assay = get_wells_by_id_assay(id_plate_constituents)

id_qpcr_datas = build_id_qpcr_datas_from_inst_data(id_plate_constituents,
                                                   id_qpcr_inst_plate)

Wall time: 6.52 ms


In [11]:
%%time
mapping = create_id_qpcr_lc_mapping(expt_plates[lc_plate])
dilutions = get_lc_dilutions(expt_plates[lc_plate])
lc_datas = build_labchip_datas_from_inst_data(
    id_plate_constituents, lc_inst_plate, mapping, assays, dilutions)

Wall time: 1.5 ms


In [16]:
%%time
qwells = wells_by_id_assay[('Kp_KPC_5.x_KPC19_KPC24',)]
lcwells = []
for qw in qwells:
    if qw in mapping:
        lcwells.append(mapping[qw])
    else:
        lcwells.append(None)

nmt = VanillaMasterTable.create_from_models(qwells,
                                           id_qpcr_plate,
                                           lcwells,
                                           lc_plate,
                                           id_plate_constituents,
                                           id_qpcr_datas,
                                           lc_datas)

Wall time: 1.5 ms


In [17]:
import pandas as pd
df = pd.DataFrame(nmt.rows)
df.loc[:, ['qPCR well', 'Ct', '∆NTC_Ct', 'Ct_Call', 'Tm Specif','Tm NS', 'Tm PD']]

,qPCR well,Ct,∆NTC_Ct,Ct_Call,Tm Specif,Tm NS,Tm PD
0,A81_E217_1_E10,24.093719,15.906281,True,True,False,False
1,A81_E217_1_F10,20.041382,19.958618,True,True,False,False
2,A81_E217_1_D09,27.445021,12.554979,True,True,False,False
3,A81_E217_1_D10,27.369135,12.630865,True,True,False,False
4,A81_E217_1_B09,32.185207,7.814793,True,True,False,False
5,A81_E217_1_F09,20.481216,19.518784,True,True,False,False
6,A81_E217_1_C10,32.999088,7.000912,True,True,False,False
7,A81_E217_1_C09,32.877289,7.122711,True,True,False,False
8,A81_E217_1_B10,31.993595,8.006405,True,True,False,False
9,A81_E217_1_E09,23.967501,16.032499,True,True,False,False
